<a href="https://colab.research.google.com/github/dmamur/elembert/blob/noPosEmbeddings/element_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re, glob,os,sys,pickle,random
from collections import defaultdict

In [ ]:
!pip install ase
!git clone https://github.com/dmamur/elembert.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.7 MB/s eta 0:00:00
Cloning into 'elementsem'...
remote: Enumerating objects: 819, done.
remote: Counting objects: 100% (374/374), done.
remote: Compressing objects: 100% (200/200), done.
remote: Total 819 (delta 187), reused 342 (delta 170), pack-reused 445
Receiving objects: 100% (819/819), 99.11 MiB | 29.08 MiB/s, done.
Resolving deltas: 100% (404/404), done.


# Load data from matbench

In [ ]:
from ase.io import read,write
from scipy.ndimage import gaussian_filter1d
from ase.neighborlist import neighbor_list
from joblib import dump, load
from sklearn.cluster import KMeans
from sklearn import decomposition
from ase import Atoms

ellst=open('/content/elementsem/models/pcakm/ellist.txt','r').read().split('\n')
print(ellst)

km = {i: load('/content/elementsem/models/pcakm/'+i+'_kmeans.pkl') for i in ellst}
pca = {i: load('/content/elementsem/models/pcakm/'+i+'_pca.pkl') for i in ellst}

def getRawInputs(types,atoms,x,v):
    i, d = neighbor_list('id', atoms, 10.0, self_interaction=False)
    rdfatoms,ntypes=[],[]
    for k,l in enumerate(atoms):
        el=types[k]
        y = np.zeros(100)
        dist = np.round(d[i==k]*10)
        a,b=np.unique(dist, return_counts=True)
        np.put(y,a.astype(int)-1,b)
        values=gaussian_filter1d(y/v,1)
        num = km[el].predict(np.nan_to_num([values],nan=0,posinf=0, neginf=0))[0]
        ntypes.append(el+str(num))#el2id[el+str(num)]
    return ntypes

['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'Ra', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Rn']


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator KMeans from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator PCA from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
from ase.build import molecule
atoms = molecule('CH3CH2NH2')
#typesLstTest = getModelInputs(train_inputs)

In [ ]:
x= np.arange(0,10,0.1)
v = np.concatenate([[1],4*np.pi/3*(x[1:]**3 - x[:-1]**3)])
typesN=getRawInputs(atoms.get_chemical_symbols(),atoms,x,v)

In [ ]:
print(typesN)

['C1', 'C5', 'N3', 'H0', 'H0', 'H0', 'H0', 'H0', 'H0', 'H0']


In [ ]:
from ase.io import read, write

In [ ]:
atoms=read('/content/elementsem/models/pcakm/Figure2a.xyz',format='xyz')
typesN1=getRawInputs(atoms.get_chemical_symbols(),atoms,x,v)
print(typesN1)

['C5', 'C3', 'C0', 'C0', 'S2', 'C3', 'C0', 'C5', 'N3', 'C0', 'C5', 'S2', 'C0', 'C3', 'O0', 'O2', 'H0', 'H0', 'H0', 'H0', 'H0', 'H0', 'H0', 'H0', 'H0', 'H0', 'H0', 'H0', 'H0']


In [ ]:
atoms=read('/content/elementsem/models/pcakm/Figure2b.xyz',format='xyz')
typesN2=getRawInputs(atoms.get_chemical_symbols(),atoms,x,v)
print(typesN2)

['Li0', 'Li0', 'Li0', 'Li0', 'Li0', 'Li0', 'Li0', 'Li0', 'Li0', 'Li0', 'Li0', 'Li0', 'Li0', 'Li0', 'Li0', 'Li0', 'Co1', 'Co1', 'O0', 'O0', 'O0', 'O0', 'O0', 'O0', 'O0', 'O0', 'O0', 'O0', 'O0', 'O0']
